In [1]:
import  numpy as np

# Часть 1

## 2.1 Сформировать порождающую матрицу линейного кода (7, 4, 3)

In [5]:
n, k, d = 7, 4, 3
I_k = np.eye(k, dtype=int)
X = np.array([
        [1, 1, 0],
        [1, 0, 1],
        [0, 1, 1],
        [1, 1, 1]
    ])
G = np.hstack((I_k,X))
print(f'порождающая матрица линейного кода (7, 4, 3):\n{G}')

порождающая матрица линейного кода (7, 4, 3):
[[1 0 0 0 1 1 0]
 [0 1 0 0 1 0 1]
 [0 0 1 0 0 1 1]
 [0 0 0 1 1 1 1]]


## 2.2 Сформировать проверочную матрицу на основе порождающей

In [14]:
I_n_k = np.eye(n - k, dtype=int)

H = np.vstack((X, I_n_k))
print(f'Проверочная матрица H:\n{H}')

Проверочная матрица H:
[[1 1 0]
 [1 0 1]
 [0 1 1]
 [1 1 1]
 [1 0 0]
 [0 1 0]
 [0 0 1]]
(7, 3)


## 2.3 Сформировать таблицу синдромов для всех однократных ошибок

In [10]:
def calculate_syndrome(E, H):
    """
    Вычисляет синдром для данного вектора ошибки и матрицы H.
    """
    return tuple((E @ H) % 2)

def build_syndrome_table(H):
    """
    Строит таблицу синдромов для матрицы H.
    """
    n = H.shape[0]
    syndromes = {}
    for i in range(n):
        E = np.zeros(n, dtype=int)
        E[i] = 1
        S = calculate_syndrome(E, H)
        syndromes[S] = tuple(E)
    return syndromes
s_t = build_syndrome_table(H)
print("Syndrome\tError vec")
for syndrome in s_t:
    print(f"{syndrome}\t{s_t[syndrome]}")

Syndrome	Error vec
(1, 1, 0)	(1, 0, 0, 0, 0, 0, 0)
(1, 0, 1)	(0, 1, 0, 0, 0, 0, 0)
(0, 1, 1)	(0, 0, 1, 0, 0, 0, 0)
(1, 1, 1)	(0, 0, 0, 1, 0, 0, 0)
(1, 0, 0)	(0, 0, 0, 0, 1, 0, 0)
(0, 1, 0)	(0, 0, 0, 0, 0, 1, 0)
(0, 0, 1)	(0, 0, 0, 0, 0, 0, 1)


## 2.4 Сформировать кодовое слово длины n из слова длины k. Внести однократную ошибку в сформированное слово. Вычислить синдром, исправить ошибку с использованием таблицы синдромов. Убедиться в правильности полученного слова


In [17]:

def encode_msg(generator_matrix, msg):
    """
    Кодирует сообщение с использованием генераторной матрицы.
    """
    return np.dot(msg, generator_matrix) % 2

def add_noise(encoded_word, error_positions):
    """Вносит ошибки в кодовое слово по указанным позициям."""
    noisy_word = encoded_word.copy()
    for pos in error_positions:
        noisy_word[pos] ^= 1  # Инвертируем бит
    return noisy_word

def correct_errors(parity_check_matrix, received_word, syndromes):
    """Исправляет ошибки в принятом кодовом слове с помощью таблицы синдромов."""
    syndrome = tuple(np.dot(parity_check_matrix.T, received_word) % 2)
    if syndrome in syndromes:
        error_pattern = np.array(syndromes[syndrome], dtype=int)
        fixed_word = (received_word + error_pattern) % 2
        return fixed_word, error_pattern
    return received_word, None

# Пример использования
original_msg = np.random.randint(0, 2, size=k)
print("Оригинальное сообщение:\n", original_msg)

# Кодирование сообщения
encoded_word = encode_msg(G, original_msg)
print("\nКодированное сообщение:\n", encoded_word)

# Внесение ошибки
pos = np.random.randint(len(encoded_word))
corrupted_word = add_noise(encoded_word, [pos])
print(f"\nСообщение с ошибкой в позиции {pos}:\n", corrupted_word)

# Исправление ошибки
corrected_word, detected_error = correct_errors(H, corrupted_word, s_t)
print("\nИсправленное сообщение:\n", corrected_word)

if detected_error is not None:
    print("\nОбнаруженная ошибка:\n", detected_error)
else:
    print("\nОшибки не обнаружено.")

Оригинальное сообщение:
 [0 1 0 0]

Кодированное сообщение:
 [0 1 0 0 1 0 1]

Сообщение с ошибкой в позиции 4:
 [0 1 0 0 0 0 1]

Исправленное сообщение:
 [0 1 0 0 1 0 1]

Обнаруженная ошибка:
 [0 0 0 0 1 0 0]
